# Mining testimonial fragments of the Holocaust

**Experience domain: rape**

### Load the necessary libraries

In [75]:
import sys; sys.path.insert(0, '..')
import itertools

In [76]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [77]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [78]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [79]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [80]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [81]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [82]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [83]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [84]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [85]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [86]:
main_node = "rape"
#delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [87]:
query = '[lemma="rape"]'

result = topic_concordancer.main(query,window=50,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  Kill

In [88]:
lemmas = ["kill","rape"]

In [89]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="kill"][]{0,50}[lemma="rape"])|([lemma="rape"][]{0,50}[lemma="kill"])


In [90]:
domain_term = "kill"

In [91]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [92]:
fragment_1 = {}
fragment_1['original_sentence'] = "Sometimes, they raped them. They killed them right away."
fragment_1['label']="Sometimes, they raped them. They killed them right away."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Sometimes%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22raped%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22They%22%5D+%5B%22killed%22%5D+%5B%22them%22%5D+%5B%22right%22%5D+%5B%22away%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Sometimes , they raped them . They killed them right away . ', 'right': '', 'complete_match': 'Sometimes , they raped them . They killed them right away . ', 'testimony_id': 'usc_shoah_2590', 'shelfmark': ['USC 2590'], 'token_start': 1782, 'token_end': 1794}


In [93]:
fragment_2 = {}
fragment_2['original_sentence'] = "The night that they marched into Jasionowka, we heard rumors, even before, that wherever they marched in, they burned, they killed, they raped, and they did horrible things to the Jewish people. "
fragment_2['label']="they burned, they killed, they raped,"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22night%22%5D+%5B%22that%22%5D+%5B%22they%22%5D+%5B%22marched%22%5D+%5B%22into%22%5D+%5B%22Jasionowka%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22heard%22%5D+%5B%22rumors%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22before%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22wherever%22%5D+%5B%22they%22%5D+%5B%22marched%22%5D+%5B%22in%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22burned%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22raped%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22did%22%5D+%5B%22horrible%22%5D+%5B%22things%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22Jewish%22%5D+%5B%22people%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The night that they marched into Jasionowka , we heard rumors , even before , that wherever they marched in , they burned , they killed , they raped , and they did horrible things to the Jewish people . ', 'right': '', 

In [94]:
fragment_3 = {}
fragment_3['original_sentence'] = "They grabbed the girl and raped her and they cry"
fragment_3['label']="They grabbed the girl and raped her and they cry"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22grabbed%22%5D+%5B%22the%22%5D+%5B%22girl%22%5D+%5B%22and%22%5D+%5B%22raped%22%5D+%5B%22her%22%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22cry%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They grabbed the girl and raped her and they cry ', 'right': '', 'complete_match': 'They grabbed the girl and raped her and they cry ', 'testimony_id': 'irn504583', 'shelfmark': ['USHMM RG-50.030*0088'], 'token_start': 16020, 'token_end': 16030}


In [95]:
fragment_4 = {}
fragment_4['original_sentence'] = "The Russian took her, raped her, and killed her."
fragment_4['label']= "The Russian took her, raped her, and killed her."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22Russian%22%5D+%5B%22took%22%5D+%5B%22her%22%5D+%5B%5D+%5B%22raped%22%5D+%5B%22her%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22killed%22%5D+%5B%22her%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'The Russian took her , raped her , and killed her . ', 'right': '', 'complete_match': 'The Russian took her , raped her , and killed her . ', 'testimony_id': 'irn508651', 'shelfmark': ['USHMM RG-50.462*0031'], 'token_start': 38455, 'token_end': 38467}


In [96]:
fragment_5 = {}
fragment_5['original_sentence'] = "And an aunt was raped and killed."
fragment_5['label']= "And an aunt was raped and killed."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22an%22%5D+%5B%22aunt%22%5D+%5B%22was%22%5D+%5B%22raped%22%5D+%5B%22and%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And an aunt was raped and killed . ', 'right': '', 'complete_match': 'And an aunt was raped and killed . ', 'testimony_id': 'irn509148', 'shelfmark': ['USHMM RG-50.233*0065'], 'token_start': 1503, 'token_end': 1511}


In [97]:
add_testimonial_fragments(fragments)

### 2.  Watch

In [98]:
lemmas = []

In [99]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

()


In [100]:
domain_term = "watch"

In [101]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [102]:
fragment_1 = {}
fragment_1['original_sentence'] = "She saw her mother raped to death"
fragment_1['label']="She saw her mother raped to death"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22She%22%5D+%5B%22saw%22%5D+%5B%22her%22%5D+%5B%22mother%22%5D+%5B%22raped%22%5D+%5B%22to%22%5D+%5B%22death%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'She saw her mother raped to death ', 'right': '', 'complete_match': 'She saw her mother raped to death ', 'testimony_id': 'usc_shoah_14900', 'shelfmark': ['USC 14900'], 'token_start': 15674, 'token_end': 15681}


In [103]:
fragment_2 = {}
fragment_2['original_sentence'] = "And I watched them rape the women."
fragment_2['label']=" And I watched them rape the women."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22watched%22%5D+%5B%22them%22%5D+%5B%22rape%22%5D+%5B%22the%22%5D+%5B%22women%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And I watched them rape the women . ', 'right': '', 'complete_match': 'And I watched them rape the women . ', 'testimony_id': 'usc_shoah_162', 'shelfmark': ['USC 162'], 'token_start': 7990, 'token_end': 7998}


In [104]:
fragment_3 = {}
fragment_3['original_sentence'] = "And to see those young women raped by the men there with sticks, and my mother was near me and she took her hand and put on my eyes, I shouldn't see for the first time sexual intercourse."
fragment_3['label']="(..)to see those young women raped by the men (..)"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%22those%22%5D+%5B%22young%22%5D+%5B%22women%22%5D+%5B%22raped%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22men%22%5D+%5B%22there%22%5D+%5B%22with%22%5D+%5B%22sticks%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22my%22%5D+%5B%22mother%22%5D+%5B%22was%22%5D+%5B%22near%22%5D+%5B%22me%22%5D+%5B%22and%22%5D+%5B%22she%22%5D+%5B%22took%22%5D+%5B%22her%22%5D+%5B%22hand%22%5D+%5B%22and%22%5D+%5B%22put%22%5D+%5B%22on%22%5D+%5B%22my%22%5D+%5B%22eyes%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22should%22%5D+%5B%5D+%5B%22see%22%5D+%5B%22for%22%5D+%5B%22the%22%5D+%5B%22first%22%5D+%5B%22time%22%5D+%5B%22sexual%22%5D+%5B%22intercourse%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And to see those young women raped by the men there with sticks , and my mother was near me and she took her hand and put on my eyes , I should n't see for the first time sexual in

In [105]:
fragment_4 = {}
fragment_4['original_sentence'] = "that's the one, she was raped"
fragment_4['label']= " She saw, she-- that's the one, she was raped"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22that%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22one%22%5D+%5B%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22raped%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "that 's the one , she was raped ", 'right': '', 'complete_match': "that 's the one , she was raped ", 'testimony_id': 'usc_shoah_1222', 'shelfmark': ['USC 1222'], 'token_start': 2189, 'token_end': 2197}


In [106]:
add_testimonial_fragments(fragments)

### 3.  

In [107]:
lemmas = ["shoot","rape"]

In [108]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="shoot"][]{0,50}[lemma="rape"])|([lemma="rape"][]{0,50}[lemma="shoot"])


In [109]:
domain_term = "shoot"

In [110]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [111]:
fragment_1 = {}
fragment_1['original_sentence'] = "They probably raped them, abused them, eventually shot them because nobody survived"
fragment_1['label']="They probably raped them, abused them, eventually shot them because nobody survived"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22probably%22%5D+%5B%22raped%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22abused%22%5D+%5B%22them%22%5D+%5B%5D+%5B%22eventually%22%5D+%5B%22shot%22%5D+%5B%22them%22%5D+%5B%22because%22%5D+%5B%22nobody%22%5D+%5B%22survived%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They probably raped them , abused them , eventually shot them because nobody survived ', 'right': '', 'complete_match': 'They probably raped them , abused them , eventually shot them because nobody survived ', 'testimony_id': 'usc_shoah_10406', 'shelfmark': ['USC 10406'], 'token_start': 20350, 'token_end': 20364}


In [112]:
fragment_2 = {}
fragment_2['original_sentence'] = "They picked the nice looking girls, they raped them and then they shot them."
fragment_2['label']="They picked the nice looking girls, they raped them and then they shot them."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22picked%22%5D+%5B%22the%22%5D+%5B%22nice%22%5D+%5B%22looking%22%5D+%5B%22girls%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22raped%22%5D+%5B%22them%22%5D+%5B%22and%22%5D+%5B%22then%22%5D+%5B%22they%22%5D+%5B%22shot%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They picked the nice looking girls , they raped them and then they shot them . ', 'right': '', 'complete_match': 'They picked the nice looking girls , they raped them and then they shot them . ', 'testimony_id': 'irn510487', 'shelfmark': ['USHMM RG-50.322*0033'], 'token_start': 34868, 'token_end': 34884}


In [113]:
fragment_3 = {}
fragment_3['original_sentence'] = "was raped and shot in the middle of the street."
fragment_3['label']="(..) name was Hannah Prusak, P-R-U-S-A-K-- was raped and shot in the middle of the street."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22was%22%5D+%5B%22raped%22%5D+%5B%22and%22%5D+%5B%22shot%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22middle%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22street%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'was raped and shot in the middle of the street . ', 'right': '', 'complete_match': 'was raped and shot in the middle of the street . ', 'testimony_id': 'usc_shoah_11593', 'shelfmark': ['USC 11593'], 'token_start': 3721, 'token_end': 3732}


In [114]:
fragment_4 = {}
fragment_4['original_sentence'] = "I know she was raped. I know that they put a big stick through her genitals. I know that they tortured her before they shot her. "
fragment_4['label']= "I know she was raped. (.. ) I know that they tortured her before they shot her."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22know%22%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22raped%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22know%22%5D+%5B%22that%22%5D+%5B%22they%22%5D+%5B%22put%22%5D+%5B%22a%22%5D+%5B%22big%22%5D+%5B%22stick%22%5D+%5B%22through%22%5D+%5B%22her%22%5D+%5B%22genitals%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22know%22%5D+%5B%22that%22%5D+%5B%22they%22%5D+%5B%22tortured%22%5D+%5B%22her%22%5D+%5B%22before%22%5D+%5B%22they%22%5D+%5B%22shot%22%5D+%5B%22her%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I know she was raped . I know that they put a big stick through her genitals . I know that they tortured her before they shot her . ', 'right': '', 'complete_match': 'I know she was raped . I know that they put a big stick through her genitals . I know that they tortured her before they shot her . ', 'testimony_id': 'usc_shoah_5799', 'shelfmark': ['USC 5799'], 'token_start': 5010, 'to

In [115]:
add_testimonial_fragments(fragments)

### 4. afraid  

In [116]:
lemmas = ["afraid","rape"]

In [117]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="afraid"][]{0,50}[lemma="rape"])|([lemma="rape"][]{0,50}[lemma="afraid"])


In [118]:
domain_term = "afraid"

In [119]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [120]:
fragment_1 = {}
fragment_1['original_sentence'] = "I did not know what rape meant, but I knew it was something that women were afraid of."
fragment_1['label']="I did not know what rape meant, but I knew it was something that women were afraid of."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22did%22%5D+%5B%22not%22%5D+%5B%22know%22%5D+%5B%22what%22%5D+%5B%22rape%22%5D+%5B%22meant%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22I%22%5D+%5B%22knew%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22something%22%5D+%5B%22that%22%5D+%5B%22women%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22of%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I did not know what rape meant , but I knew it was something that women were afraid of . ', 'right': '', 'complete_match': 'I did not know what rape meant , but I knew it was something that women were afraid of . ', 'testimony_id': 'HVT-123', 'shelfmark': ['Fortunoff HVT-123'], 'token_start': 12222, 'token_end': 12242}


In [121]:
fragment_2 = {}
fragment_2['original_sentence'] = "you know, that they would come and rape us, or whatever."
fragment_2['label']="And these were soldiers and they were afraid of -- you know, that they would come and rape us, or whatever."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22they%22%5D+%5B%22would%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22rape%22%5D+%5B%22us%22%5D+%5B%5D+%5B%22or%22%5D+%5B%22whatever%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'you know , that they would come and rape us , or whatever . ', 'right': '', 'complete_match': 'you know , that they would come and rape us , or whatever . ', 'testimony_id': 'irn506747', 'shelfmark': ['USHMM RG-50.549.02*0020'], 'token_start': 6476, 'token_end': 6490}


In [122]:
fragment_3 = {}
fragment_3['original_sentence'] = "We were afraid they were going to rape us."
fragment_3['label']="We were afraid they were going to rape us."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22rape%22%5D+%5B%22us%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were afraid they were going to rape us . ', 'right': '', 'complete_match': 'We were afraid they were going to rape us . ', 'testimony_id': 'irn504676', 'shelfmark': ['USHMM RG-50.030*0176'], 'token_start': 15013, 'token_end': 15023}


In [123]:
fragment_4 = {}
fragment_4['original_sentence'] = "We were afraid for raping."
fragment_4['label']= "We were afraid for raping."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22for%22%5D+%5B%22raping%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were afraid for raping . ', 'right': '', 'complete_match': 'We were afraid for raping . ', 'testimony_id': 'irn504842', 'shelfmark': ['USHMM RG-50.030*0348'], 'token_start': 18646, 'token_end': 18652}


In [124]:
add_testimonial_fragments(fragments)